In [3]:
# !pip install nflfastpy
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install requests

In [15]:
#using nfl-env conda python 3.10.4 environment 
import pandas as pd
import numpy as np
import nflfastpy as nfl 
from matplotlib import pyplot as plt
from matplotlib import style
import requests
from io import BytesIO

pd.options.display.max_rows = 100
pd.options.display.max_columns = 1000
style.use("ggplot")
style.use("seaborn-notebook")
style.use("fivethirtyeight")

In [89]:
df2018 = nfl.load_pbp_data(2018)
df2018['year'] = 2018
df2019 = nfl.load_pbp_data(2019)
df2019['year'] = 2019
df2020 = nfl.load_pbp_data(2020) 
df2020['year'] = 2020
df2021 = nfl.load_pbp_data(2021)
df2021['year'] = 2021

In [90]:
dfTotal = pd.concat([df2018, df2019, df2020, df2021])

####columns i don't know
# sp
#


fieldsIcareAbout = ['play_id', 'game_id', 'home_team', 'away_team', 'season_type', 'week', 
                    'posteam', 'defteam', 'side_of_field', 'yardline_100', 'game_date', 
                    'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining'
                    'game_half', 'quarter_end', 'drive', 'sp', 'qtr', 'down', 'goal_to_go', 
                    'time', 'yrdln', 'ydstogo', 'ydsnet', 'desc', 'play_type', 'yards_gained',
                    'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 
                    'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 
                    'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_poin_conv_result',
                    'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout', 'timeout_team', 
                    'td_team', 'td_player_name', 'td_player_id', 'total_home_score', 'total_away_score',
                    'posteam_score', 'defteam_score', 'score_differential', 'posteam_score_post', 'defteam_score_post',
                    'score_differential_post', 'punt_blocked', 'first_down_rush', 'first_down_pass', 'first_down_penalty', 
                    'thirty_down_converted', 'third_down_failed', 'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass', 
                    'touchback', 'interception', 'punt_inside_twenty', 'punt_in_endzone', 'punt_out_of_bounds', 'punt_downed', 
                    'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone', 'kickoff_out_of_bounds', 'kickoff_downed', 
                    'kickoff_fair_catch', 'fumble_forced', 'fumble_not_forced', 'fumble_out_of_bounds', 'solo_tackle', 'safety', 
                    'penalty', 'tackled_for_loss', 'fumble_lost', 'own_kickoff_recovery', 'own_kickoff_recovery_td', 'qb_hit', 
                    'rush_attempt', 'pass_attempt', 'sack', 'touchdown', 'pass_touchdown', 'rush_touchdown', 'return_touchdown', 
                    'extra_point_attempt', 'two_point_attempt', 'field_goal_attempt', 'kickoff_attempt', 'punt_attempt', 'fumble', 
                    'complete_pass', 'assist_tackle', 'lateral_reception', 'lateral_rush', 'lateral_return', 'lateral_recovery', 
                    'passer_player_id', 'passer_player_name', 'passing_yards', 'receiver_player_id', 'receiver_player_name', 
                    'receiving_yards', 'rusher_player_id', 'rusher_player_name', 'rushing_yards', 'later_receiver_player_name', 
                    'lateral_receiving_yards', 'later_rusher_player_id', 'lateral_rusher_player_name', 'lateral_rushing_yards', 
                    'lateral_sack_player_id', 'lateral_sack_player_name', 'interception_player_id', 'interception_player_name', 
                    'lateral_interception_player_id', 'lateral_interception_player_name', 'tackle_for_loss_1_player_id', 'tackle_for_loss_1_player_name',
                    'tackle_for_loss_2_player_id', 'tackle_for_loss_2_player_name', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'special_teams_play', 
                    'out_of_bounds', 
                    ]
                    
                    ###Special Teams 
                    # 'punt_returner_player_id', 'punt_returner_player_name',
                    # 'lateral' ]




In [91]:
rushing_stats = ['year', 'yardline_100', 'down', 'ydstogo', 'run_location', 'defteam', 'fantasy_player_id', 'fantasy_player_name', 'roof',  'surface', 'temp', 'wind', 'score_differential', 'goal_to_go', 'rushing_yards']
rush_filter = dfTotal['play_type'] == 'run'
goalFilter = dfTotal['goal_to_go'] == 0 
dfRushing = dfTotal[rush_filter&goalFilter][rushing_stats].copy(deep=True)
dfRushing['rush_attempts'] = 1

In [92]:
rushing_grouped = dfRushing.groupby(['fantasy_player_name', 'year'], as_index=False).agg({'rushing_yards' : sum, 'rush_attempts' : sum })

rushing_grouped['ypc'] = rushing_grouped['rushing_yards']/rushing_grouped['rush_attempts'] 

In [93]:
rushing_grouped[rushing_grouped['rush_attempts'] >= 100].pivot_table(index='fantasy_player_name', columns='year', values='ypc', aggfunc=sum).sort_values(by=2018, ascending=False).dropna()

#rushing_grouped.sort_values(by='ypc', ascending=False, inplace=True)

year,2018,2019,2020,2021
fantasy_player_name,,,,
L.Jackson,5.620155,7.964706,7.122699,6.352459
A.Jones,5.543307,4.743902,5.935000,4.800000
N.Chubb,5.463277,5.496296,5.842365,5.783019
D.Henry,5.293814,5.348066,5.506812,4.234513
A.Ekeler,5.272727,4.504132,4.654206,4.620321
J.Mixon,5.192661,4.324219,3.681416,4.220238
M.Gordon,4.844086,4.000000,4.700000,4.800000
J.Conner,4.840206,4.172727,4.358824,4.050279
D.Cook,4.818898,4.671937,5.278169,4.956522
